# LLM Evaluation with MLflow Example Notebook

In this notebook, we will demonstrate how to evaluate various LLMs and RAG systems with MLflow, leveraging simple metrics such as toxicity, as well as LLM-judged metrics such as relevance, and even custom LLM-judged metrics such as professionalism

We need to set our OpenAI API key, since we will be using GPT-4 for our LLM-judged metrics.

In order to set your private key safely, please be sure to either export your key through a command-line terminal for your current instance, or, for a permanent addition to all user-based sessions, configure your favored environment management configuration file (i.e., .bashrc, .zshrc) to have the following entry:

`OPENAI_API_KEY=<your openai API key>`

In [1]:
import os

import mlflow
import openai
import pandas as pd
from dotenv import load_dotenv

## More Setup

In [2]:
load_dotenv()

print('OPENAI_API_KEY=' + os.getenv('OPENAI_API_KEY')[:20] + '...')

OPENAI_API_KEY=sk-proj-ZhcCqw0r70dP...


## Basic Question-Answering Evaluation

Create a test case of `inputs` that will be passed into the model and `ground_truth` which will be used to compare against the generated output from the model.

In [3]:
eval_df = pd.DataFrame(
    {
        "inputs": [
            "How does useEffect() work?",
            "What does the static keyword in a function mean?",
            "What does the 'finally' block in Python do?",
            "What is the difference between multiprocessing and multithreading?",
        ],
        "ground_truth": [
            "The useEffect() hook tells React that your component needs to do something after render. React will remember the function you passed (we’ll refer to it as our “effect”), and call it later after performing the DOM updates.",
            "Static members belongs to the class, rather than a specific instance. This means that only one instance of a static member exists, even if you create multiple objects of the class, or if you don't create any. It will be shared by all objects.",
            "'Finally' defines a block of code to run when the try... except...else block is final. The finally block will be executed no matter if the try block raises an error or not.",
            "Multithreading refers to the ability of a processor to execute multiple threads concurrently, where each thread runs a process. Whereas multiprocessing refers to the ability of a system to run multiple processors in parallel, where each processor can run one or more threads.",
        ],
    }
)

Create a simple OpenAI model that asks GPT to answer the question in two sentences. Call `mlflow.evaluate()` with the model and evaluation dataframe.

In [12]:
mlflow.set_experiment("openai-notebook")

with mlflow.start_run() as run:
    system_prompt = "Answer the following question in two sentences"
    basic_qa_model = mlflow.openai.log_model(
        model="gpt-4.1-mini",
        task=openai.chat.completions,
        name="model",
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": "{question}"},
        ],
    )
    results = mlflow.evaluate(
        basic_qa_model.model_uri,
        eval_df,
        targets="ground_truth",  # specify which column corresponds to the expected output
        model_type="question-answering",  # model type indicates which metrics are relevant for this task
        evaluators="default",
    )
results.metrics

2025/06/25 21:57:20 INFO mlflow.tracking.fluent: Active model is set to the logged model with ID: m-9ad2531329c241639cfdef7861fe95a1
2025/06/25 21:57:20 INFO mlflow.tracking.fluent: Use `mlflow.set_active_model` to set the active model to a different one if needed.
2025/06/25 21:57:20 INFO mlflow.tracking.fluent: Active model is set to the logged model with ID: m-9ad2531329c241639cfdef7861fe95a1
2025/06/25 21:57:20 INFO mlflow.tracking.fluent: Use `mlflow.set_active_model` to set the active model to a different one if needed.
2025/06/25 21:57:20 INFO mlflow.models.evaluation.evaluators.default: Computing model predictions.
2025/06/25 21:57:22 INFO mlflow.models.evaluation.default_evaluator: Testing metrics on first row...
2025/06/25 21:57:22 WARNING mlflow.metrics.metric_definitions: Failed to load 'toxicity' metric (error: ModuleNotFoundError("No module named 'evaluate'")), skipping metric logging.
2025/06/25 21:57:22 WARNING mlflow.models.evaluation.utils.metric: Did not log metric '

🏃 View run bald-rook-19 at: http://localhost:5001/#/experiments/2/runs/a97ab7d3d4ac46dbbc8396e3edaa79de
🧪 View experiment at: http://localhost:5001/#/experiments/2


{'exact_match/v1': 0.0}

[Trace(trace_id=21ee2ac962b5459ca0f0a4327e49c866), Trace(trace_id=d8b63c0a8fa34c8a963ffd72ad54a40e), Trace(trace_id=c69de300f2e7472ba4bf7710babfe376), Trace(trace_id=a2922e382bd34c7aac2ad6ede1256090)]

Inspect the evaluation results table as a dataframe to see row-by-row metrics to further assess model performance

In [13]:
results.tables["eval_results_table"]

,inputs,ground_truth,outputs,token_count
0,How does useEffect() work?,The useEffect() hook tells React that your com...,useEffect() is a React hook that runs side eff...,69
1,What does the static keyword in a function mean?,"Static members belongs to the class, rather th...",The static keyword in a function means the fun...,47
2,What does the 'finally' block in Python do?,'Finally' defines a block of code to run when ...,The 'finally' block in Python contains code th...,44
3,What is the difference between multiprocessing...,Multithreading refers to the ability of a proc...,Multiprocessing involves running multiple proc...,86


## LLM-judged correctness with OpenAI GPT-4

In [14]:
from mlflow.metrics.genai import EvaluationExample, answer_similarity

# Create an example to describe what answer_similarity means like for this problem.
example = EvaluationExample(
    input="What is MLflow?",
    output="MLflow is an open-source platform for managing machine "
           "learning workflows, including experiment tracking, model packaging, "
           "versioning, and deployment, simplifying the ML lifecycle.",
    score=4,
    justification="The definition effectively explains what MLflow is "
                  "its purpose, and its developer. It could be more concise for a 5-score.",
    grading_context={
        "targets": "MLflow is an open-source platform for managing "
                   "the end-to-end machine learning (ML) lifecycle. It was developed by Databricks, "
                   "a company that specializes in big data and machine learning solutions. MLflow is "
                   "designed to address the challenges that data scientists and machine learning "
                   "engineers face when developing, training, and deploying machine learning models."
    },
)

# Construct the metric using an OpenAI model as the judge
answer_similarity_metric = answer_similarity(model="openai:/gpt-4.1-mini", examples=[example])

print(answer_similarity_metric)

EvaluationMetric(name=answer_similarity, greater_is_better=True, long_name=answer_similarity, version=v1, metric_details=
Task:
You must return the following fields in your response in two lines, one below the other:
score: Your numerical score for the model's answer_similarity based on the rubric
justification: Your reasoning about the model's answer_similarity score

You are an impartial judge. You will be given an input that was sent to a machine
learning model, and you will be given an output that the model produced. You
may also be given additional information that was used by the model to generate the output.

Your task is to determine a numerical score called answer_similarity based on the input and output.
A definition of answer_similarity and a grading rubric are provided below.
You must use the grading rubric to determine your score. You must also justify your score.

Examples could be included below for reference. Make sure to use them as references and to
understand them be

Call `mlflow.evaluate()` again but with your new `answer_similarity_metric`

In [15]:
with mlflow.start_run() as run:
    results = mlflow.evaluate(
        basic_qa_model.model_uri,
        eval_df,
        targets="ground_truth",
        model_type="question-answering",
        evaluators="default",
        extra_metrics=[answer_similarity_metric],  # use the answer similarity metric created above
    )
results.metrics

2025/06/25 21:57:39 INFO mlflow.tracking.fluent: Active model is set to the logged model with ID: m-9ad2531329c241639cfdef7861fe95a1
2025/06/25 21:57:39 INFO mlflow.tracking.fluent: Use `mlflow.set_active_model` to set the active model to a different one if needed.
2025/06/25 21:57:39 INFO mlflow.models.evaluation.evaluators.default: Computing model predictions.
2025/06/25 21:57:41 INFO mlflow.models.evaluation.default_evaluator: Testing metrics on first row...
2025/06/25 21:57:41 WARNING mlflow.metrics.metric_definitions: Failed to load 'toxicity' metric (error: ModuleNotFoundError("No module named 'evaluate'")), skipping metric logging.
2025/06/25 21:57:41 WARNING mlflow.models.evaluation.utils.metric: Did not log metric 'toxicity' at index 1 in the `extra_metrics` parameter because it returned None.
2025/06/25 21:57:41 WARNING mlflow.metrics.metric_definitions: Failed to import textstat for flesch kincaid metric, skipping metric logging. Please install textstat using 'pip install te

🏃 View run big-shad-953 at: http://localhost:5001/#/experiments/2/runs/dd4ce1f12c104094b7349452d92d6499
🧪 View experiment at: http://localhost:5001/#/experiments/2


{'exact_match/v1': 0.0,
 'answer_similarity/v1/mean': np.float64(4.0),
 'answer_similarity/v1/variance': np.float64(1.5),
 'answer_similarity/v1/p90': np.float64(5.0)}

[Trace(trace_id=3008983d6f3a4dd5851ad5ed2571f3cb), Trace(trace_id=4ee3af6aff1541d5a5cbcc78499f3be4), Trace(trace_id=72a1fa7213134983b1ab7ceb8f8b81f0), Trace(trace_id=7232b0717fd147adaed70ee5b849e024)]

See the row-by-row LLM-judged answer similarity score and justifications

In [16]:
results.tables["eval_results_table"]

,inputs,ground_truth,outputs,token_count,answer_similarity/v1/score,answer_similarity/v1/justification
0,How does useEffect() work?,The useEffect() hook tells React that your com...,useEffect() is a React hook that runs side eff...,68,5,The output closely aligns with the target by a...
1,What does the static keyword in a function mean?,"Static members belongs to the class, rather th...",The static keyword in a function means that th...,54,2,The output discusses the static keyword in fun...
2,What does the 'finally' block in Python do?,'Finally' defines a block of code to run when ...,The 'finally' block in Python contains code th...,45,5,The output fully captures the key aspects of t...
3,What is the difference between multiprocessing...,Multithreading refers to the ability of a proc...,Multiprocessing involves running multiple proc...,74,4,The output explains multiprocessing and multit...


## Custom LLM-judged metric for professionalism

Create a custom metric that will be used to determine professionalism of the model outputs. Use `make_genai_metric` with a metric definition, grading prompt, grading example, and judge model configuration

In [17]:
from mlflow.metrics.genai import EvaluationExample, make_genai_metric

professionalism_metric = make_genai_metric(
    name="professionalism",
    definition=(
        "Professionalism refers to the use of a formal, respectful, and appropriate style of communication that is tailored to the context and audience. It often involves avoiding overly casual language, slang, or colloquialisms, and instead using clear, concise, and respectful language"
    ),
    grading_prompt=(
        "Professionalism: If the answer is written using a professional tone, below "
        "are the details for different scores: "
        "- Score 1: Language is extremely casual, informal, and may include slang or colloquialisms. Not suitable for professional contexts."
        "- Score 2: Language is casual but generally respectful and avoids strong informality or slang. Acceptable in some informal professional settings."
        "- Score 3: Language is balanced and avoids extreme informality or formality. Suitable for most professional contexts. "
        "- Score 4: Language is noticeably formal, respectful, and avoids casual elements. Appropriate for business or academic settings. "
        "- Score 5: Language is excessively formal, respectful, and avoids casual elements. Appropriate for the most formal settings such as textbooks. "
    ),
    examples=[
        EvaluationExample(
            input="What is MLflow?",
            output=(
                "MLflow is like your friendly neighborhood toolkit for managing your machine learning projects. It helps you track experiments, package your code and models, and collaborate with your team, making the whole ML workflow smoother. It's like your Swiss Army knife for machine learning!"
            ),
            score=2,
            justification=(
                "The response is written in a casual tone. It uses contractions, filler words such as 'like', and exclamation points, which make it sound less professional. "
            ),
        )
    ],
    version="v1",
    model="openai:/gpt-4.1-mini",
    parameters={"temperature": 0.0},
    grading_context_columns=[],
    aggregations=["mean", "variance", "p90"],
    greater_is_better=True,
)

print(professionalism_metric)

EvaluationMetric(name=professionalism, greater_is_better=True, long_name=professionalism, version=v1, metric_details=
Task:
You must return the following fields in your response in two lines, one below the other:
score: Your numerical score for the model's professionalism based on the rubric
justification: Your reasoning about the model's professionalism score

You are an impartial judge. You will be given an input that was sent to a machine
learning model, and you will be given an output that the model produced. You
may also be given additional information that was used by the model to generate the output.

Your task is to determine a numerical score called professionalism based on the input and output.
A definition of professionalism and a grading rubric are provided below.
You must use the grading rubric to determine your score. You must also justify your score.

Examples could be included below for reference. Make sure to use them as references and to
understand them before complet

Call `mlflow.evaluate` with your new professionalism metric. 

In [18]:
with mlflow.start_run() as run:
    results = mlflow.evaluate(
        basic_qa_model.model_uri,
        eval_df,
        model_type="question-answering",
        evaluators="default",
        extra_metrics=[professionalism_metric],  # use the professionalism metric we created above
    )
print(results.metrics)

2025/06/25 21:58:08 INFO mlflow.tracking.fluent: Active model is set to the logged model with ID: m-9ad2531329c241639cfdef7861fe95a1
2025/06/25 21:58:08 INFO mlflow.tracking.fluent: Use `mlflow.set_active_model` to set the active model to a different one if needed.
2025/06/25 21:58:08 INFO mlflow.models.evaluation.evaluators.default: Computing model predictions.
2025/06/25 21:58:09 INFO mlflow.models.evaluation.default_evaluator: Testing metrics on first row...
2025/06/25 21:58:09 WARNING mlflow.metrics.metric_definitions: Failed to load 'toxicity' metric (error: ModuleNotFoundError("No module named 'evaluate'")), skipping metric logging.
2025/06/25 21:58:09 WARNING mlflow.models.evaluation.utils.metric: Did not log metric 'toxicity' at index 1 in the `extra_metrics` parameter because it returned None.
2025/06/25 21:58:09 WARNING mlflow.metrics.metric_definitions: Failed to import textstat for flesch kincaid metric, skipping metric logging. Please install textstat using 'pip install te

🏃 View run marvelous-ox-497 at: http://localhost:5001/#/experiments/2/runs/5bc336ee578d49cc83c4f13f7dafac00
🧪 View experiment at: http://localhost:5001/#/experiments/2
{'professionalism/v1/mean': np.float64(4.0), 'professionalism/v1/variance': np.float64(0.0), 'professionalism/v1/p90': np.float64(4.0)}


[Trace(trace_id=7317cc11742f4432bdad04bd175bf8cc), Trace(trace_id=843690b7eb334ed09d0e86fc10835b05), Trace(trace_id=82d580120e9f4463bbfe3f7783af7c40), Trace(trace_id=7e0e70639c8a4fb984f0c7deb559ac3c)]

In [20]:
results.tables["eval_results_table"]

,inputs,ground_truth,outputs,token_count,professionalism/v1/score,professionalism/v1/justification
0,How does useEffect() work?,The useEffect() hook tells React that your com...,useEffect() is a React hook that runs side eff...,58,4,"The response uses clear, formal, and respectfu..."
1,What does the static keyword in a function mean?,"Static members belongs to the class, rather th...",The static keyword in a function means that th...,57,4,"The response uses clear, formal, and respectfu..."
2,What does the 'finally' block in Python do?,'Finally' defines a block of code to run when ...,The `finally` block in Python contains code th...,44,4,"The response uses formal and clear language, a..."
3,What is the difference between multiprocessing...,Multithreading refers to the ability of a proc...,Multiprocessing involves running multiple proc...,71,4,"The response uses formal and clear language, a..."


Lets see if we can improve `basic_qa_model` by creating a new model that could perform better by changing the system prompt.

Call `mlflow.evaluate()` using the new model. Observe that the professionalism score has increased!

In [21]:
with mlflow.start_run() as run:
    system_prompt = "Answer the following question using extreme formality."
    professional_qa_model = mlflow.openai.log_model(
        model="gpt-4o-mini",
        task=openai.chat.completions,
        name="model",
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": "{question}"},
        ],
    )
    results = mlflow.evaluate(
        professional_qa_model.model_uri,
        eval_df,
        model_type="question-answering",
        evaluators="default",
        extra_metrics=[professionalism_metric],
    )
print(results.metrics)

2025/06/25 21:58:40 INFO mlflow.tracking.fluent: Active model is set to the logged model with ID: m-3759b771df184e0bba68d2468ae9b1da
2025/06/25 21:58:40 INFO mlflow.tracking.fluent: Use `mlflow.set_active_model` to set the active model to a different one if needed.
2025/06/25 21:58:40 INFO mlflow.tracking.fluent: Active model is set to the logged model with ID: m-3759b771df184e0bba68d2468ae9b1da
2025/06/25 21:58:40 INFO mlflow.tracking.fluent: Use `mlflow.set_active_model` to set the active model to a different one if needed.
2025/06/25 21:58:40 INFO mlflow.models.evaluation.evaluators.default: Computing model predictions.
2025/06/25 21:58:47 INFO mlflow.models.evaluation.default_evaluator: Testing metrics on first row...
2025/06/25 21:58:47 WARNING mlflow.metrics.metric_definitions: Failed to load 'toxicity' metric (error: ModuleNotFoundError("No module named 'evaluate'")), skipping metric logging.
2025/06/25 21:58:47 WARNING mlflow.models.evaluation.utils.metric: Did not log metric '

🏃 View run delightful-frog-372 at: http://localhost:5001/#/experiments/2/runs/76b0b50e0f1042588b60573281975bc9
🧪 View experiment at: http://localhost:5001/#/experiments/2
{'professionalism/v1/mean': np.float64(5.0), 'professionalism/v1/variance': np.float64(0.0), 'professionalism/v1/p90': np.float64(5.0)}


[Trace(trace_id=7b335e17c9d1428bb9dbd0a90e44c826), Trace(trace_id=29bc2722c97c4149afdd8556290860cf), Trace(trace_id=62d891f46b394cf987bde9a7b876afc8), Trace(trace_id=335ecf7298924a4aaa08c03c9731c6f3)]

In [14]:
results.tables["eval_results_table"]

,inputs,ground_truth,outputs,token_count,toxicity/v1/score,flesch_kincaid_grade_level/v1/score,ari_grade_level/v1/score,professionalism/v1/score,professionalism/v1/justification
0,How does useEffect() work?,The useEffect() hook tells React that your com...,"Certainly, I shall elucidate the mechanics of ...",386,0.000398,16.3,19.7,5,The response is written in an excessively form...
1,What does the static keyword in a function mean?,"Static members belongs to the class, rather th...",The static keyword utilized in the context of ...,73,0.000143,16.4,20.0,4,The language used in the output is formal and ...
2,What does the 'finally' block in Python do?,'Finally' defines a block of code to run when ...,The 'finally' block in Python serves as an int...,97,0.000313,20.5,24.5,4,The language used in the output is formal and ...
3,What is the difference between multiprocessing...,Multithreading refers to the ability of a proc...,Allow me to elucidate upon the distinction bet...,324,0.000361,17.3,21.5,5,The response is written in an excessively form...
